# Load RL Policy

---

## Define Policy

### Policy Options

- [Documentation](https://docs.ray.io/en/latest/rllib/rllib-saving-and-loading-algos-and-policies.html)
- `policy_dir` is the path to the last version of our policy and its weights
- `checkpoint_dir` is the path to the last checkpoint from our training process. 
- We won't be using `checkpoint_dir`, but it's useful for restoring to the previous state and configuration, allowing us to continue training if needed.

<span style="color:#ef4444">
&#x2B55; When loading in these checkpoints it's best to use the same version of python used in the training environment.
</span>

In [7]:
checkpoint_dir = "sumo_3d_demo/ray_results/Sheridan/PPO/PPO_Sheridan_0c5d9_00000_0_2024-11-26_15-42-31/checkpoint_000018"

policy_dir = "sumo_3d_demo/ray_results/Sheridan/PPO/PPO_Sheridan_0c5d9_00000_0_2024-11-26_15-42-31/checkpoint_000018/policies/default_policy"

### How to load any policy

In [2]:
from ray.rllib.policy.policy import Policy

policy = Policy.from_checkpoint(policy_dir)

print(policy)

PPOTorchPolicy


### How to load any Algorithm checkpoint

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm
algorithm = Algorithm.from_checkpoint(checkpoint_dir)

---

## Load PPO policy for SUMO


### Explanation of the Code

1. **Import Required Libraries**:  
   - `sumo_rl`: Used to interact with the SUMO simulation environment.  
   - `ray.rllib.policy`: For loading and using a pre-trained RL policy.  
   - `numpy`: Handles numerical operations like array transformations.  

2. **Load Pre-trained Policy**:  
   - The policy is loaded from our last checkpoint using `Policy.from_checkpoint(policy_dir)`.

3. **Initialize SUMO-RL Environment**:  
   - The SUMO simulation requires a network file (`osm.net.xml`) and a route file (`osm.rou.xml`).  
   - GUI is enabled, and the simulation runs for up to 80,000 seconds.

4. **Environment Reset**:  
   - The environment is reset using `env.reset()`, providing initial observations (`observations`) for all agents.

5. **Action Loop**:  
   - While agents are active in the environment:  
     - **Action Computation**: For each agent:  
       - Observations are converted to a NumPy array (`obs_array`) for compatibility.  
       - The pre-trained policy computes actions based on observations using `policy.compute_single_action()`.  
     - **Environment Step**:  
       - The computed actions are passed to the environment using `env.step(actions)`, which returns:  
         - `observations`: New state information for agents.  
         - `rewards`: Immediate rewards for actions.  
         - `terminations` and `truncations`: Flags indicating if agents have completed or are truncated.  
         - `infos`: Additional environment data.  

6. **Reward Logging**:  
   - Rewards for each agent are printed for monitoring performance.


In [ ]:
import sumo_rl
import numpy as np
from ray.rllib.policy.policy import Policy

policy = Policy.from_checkpoint(policy_dir)

# Initialize SUMO-RL environment
env = sumo_rl.parallel_env(
    net_file='sumo_3d_demo/osm.net.xml',
    route_file='sumo_3d_demo/osm.rou.xml',
    use_gui=True,
    num_seconds=80000
)

# Reset the environment
observations, infos = env.reset()

while env.agents:
    # Compute actions using the loaded policy
    actions = {}
    for agent, obs in observations.items():
        # Convert observation to appropriate format
        obs_array = np.array(obs).astype(np.float32)
        
        # Compute the action using the policy
        # Extract only the action
        action, _, _ = policy.compute_single_action(obs_array) 

        actions[agent] = action
    
    # Step the environment
    observations, rewards, terminations, truncations, infos = env.step(actions)
    
    # Optional: Log rewards or other metrics
    for agent, reward in rewards.items():
        print(f"Agent: {agent}, Reward: {reward}")
